In [20]:
import yfinance as yf
import pandas as pd
import numpy as np
import random
import pickle

In [2]:
%matplotlib inline
import quantstats as qs
qs.extend_pandas()

In [3]:
industry_stocks = {'Materials': ['BHP.AX',
  'FMG.AX',
  'RIO.AX',
  'NCM.AX',
  'JHX.AX',
  'S32.AX',
  'AMC.AX',
  'NST.AX',
  'BSL.AX'],
 'Real Estate': ['GMG.AX', 'SCG.AX', 'MGR.AX', 'DXS.AX', 'SGP.AX'],
 'Financials': ['CBA.AX',
  'WBC.AX',
  'NAB.AX',
  'ANZ.AX',
  'MQG.AX',
  'QBE.AX',
  'ASX.AX',
  'SUN.AX',
  'IAG.AX'],
 'Industrials': ['TCL.AX', 'SYD.AX', 'BXB.AX', 'REH.AX', 'AIA.AX'],
 'Health Care': ['CSL.AX', 'SHL.AX', 'FPH.AX', 'COH.AX', 'RHC.AX', 'RMD.AX'],
 'Information Technology': ['APT.AX', 'XRO.AX', 'CPU.AX']}

In [4]:
stock_list =[]
for ind in industry_stocks.values():
    stock_list.extend(ind)

In [6]:
stock_dr = {}
# fetch the daily returns for each stock
for stock in stock_list:
    stock_dr[stock] = qs.utils.download_returns(stock)

In [7]:
df_ratios= pd.DataFrame(index=stock_list)
for tick, stock in stock_dr.items():
    df_ratios.loc[tick,'sharpe'] = stock.sharpe()
    df_ratios.loc[tick,'common_sense_ratio']  = stock.common_sense_ratio()
    df_ratios.loc[tick,'conditional_value_at_risk']  = stock.conditional_value_at_risk()
    df_ratios.loc[tick,'expected_return'] = stock.expected_return()
    df_ratios.loc[tick,'kelly_criterion'] = stock.kelly_criterion()
    df_ratios.loc[tick,'kurtosis'] = stock.kurtosis()
    df_ratios.loc[tick,'volatility'] = stock.volatility()
    df_ratios.loc[tick,'avg_return'] = stock.avg_return()
    #df_ratios.loc[tick,'risk_of_ruin'] = stock.risk_of_ruin()
    df_ratios.loc[tick,'sortino'] = stock.sortino()
    df_ratios.loc[tick,'volatility'] = stock.volatility()
    df_ratios.loc[tick,'profit_factor'] = stock.profit_factor()
    


* 'sharpe' - greater better
* 'common_sense_ratio' -  Above 1: make money
* 'conditional_value_at_risk' - lower
* 'expected_return' - higher
* 'kelly_criterion' - 0.05 take a 5% position 
* 'kurtosis' - high - occasional higher
* 'volatility' - lower
* 'avg_return' - higher
* 'risk_of_ruin' - lower
* 'sortino' - greater better
* 'volatility' - lower
* profit_factor  - higher better

In [8]:
ratio_def= {'sharpe' : "higher",
'common_sense_ratio' :  "Above 1",
'conditional_value_at_risk' :  "lower",
'expected_return' :  "higher",
'kelly_criterion' :  "position size",
'kurtosis' :  "low",
'volatility' :  "lower",
'avg_return' :  "higher",
'sortino' :  "higher",
'volatility' :  "lower",
'profit_factor' :  'higher' }

In [9]:
df_ratios.columns

Index(['sharpe', 'common_sense_ratio', 'conditional_value_at_risk',
       'expected_return', 'kelly_criterion', 'kurtosis', 'volatility',
       'avg_return', 'sortino', 'profit_factor'],
      dtype='object')

In [10]:
#df_sorted_ratios = pd.DataFrame(np.sort(df_ratios.values, axis=0), index=df_ratios.index, columns=df_ratios.columns)

In [11]:
#df_sorted_ratios[df_sorted_ratios["common_sense_ratio"] > 1]

In [13]:
dict_ratio_stocks= {}
dict_ratio_stocks2= {}
for col in df_ratios.columns:
    df_sorted= df_ratios.sort_values(by=[col])
    if ratio_def[col] == "higher":
        dict_ratio_stocks[col] = df_sorted[col].tail(10)
    elif ratio_def[col] == "lower":
         dict_ratio_stocks[col] = df_sorted[col].head(10)
    elif col == "common_sense_ratio":
         dict_ratio_stocks[col] = df_sorted[df_sorted[col] > 1]['common_sense_ratio'].tail(10)



In [14]:
all_portfolio =[]
for ratio in dict_ratio_stocks.keys():
    tickers= dict_ratio_stocks[ratio].index.to_list()
    all_portfolio.extend(tickers)

In [15]:
set_port2 = set(all_portfolio)

In [16]:
len(set_port2)

28

In [17]:
ratio_based_port = df_ratios.loc[set_port2].sort_values(by="kelly_criterion",ascending=False).head(8).index.to_list()

In [18]:
ratio_based_port

['APT.AX',
 'REH.AX',
 'XRO.AX',
 'CSL.AX',
 'FMG.AX',
 'AIA.AX',
 'CBA.AX',
 'COH.AX']

In [22]:
with open('./data/pickles/ratio_based_port.pickle', 'wb') as handle:
    pickle.dump(ratio_based_port, handle, protocol=pickle.HIGHEST_PROTOCOL)